<a href="https://colab.research.google.com/github/urenajose/DS-Unit-1-Sprint-2-Statistics/blob/master/Jose_Urena_DSPT8_122_Chi2_Tests_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment - Practice Chi^2 Tests

Use the following dataset relating to math scores of students in two different Portugese schools:

<https://archive.ics.uci.edu/ml/datasets/Student+Performance>

### 1) Load the dataset specifically relating to math scores as a new dataframe.

There are two datasets in the `student.zip` file, make sure you use `student-mat.csv`.


## Imports

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind
from scipy import stats
import copy

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
pd.set_option('display.expand_frame_repr', False)

## Pandas read

In [3]:
# YOUR WORK HERE
student_df = pd.read_csv("/content/student-mat.csv", sep=";")

## Data review and Wrangling

In [4]:
print(student_df.shape)
print(student_df.head())
print(student_df.tail())

(395, 33)
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  reason guardian  traveltime  studytime  failures schoolsup famsup paid activities nursery higher internet romantic  famrel  freetime  goout  Dalc  Walc  health  absences  G1  G2  G3
0     GP   F   18       U     GT3       A     4     4  at_home   teacher  course   mother           2          2         0       yes     no   no         no     yes    yes       no       no       4         3      4     1     1       3         6   5   6   6
1     GP   F   17       U     GT3       T     1     1  at_home     other  course   father           1          2         0        no    yes   no         no      no    yes      yes       no       5         3      3     1     1       3         4   5   5   6
2     GP   F   15       U     LE3       T     1     1  at_home     other   other   mother           1          2         3       yes     no  yes         no     yes    yes      yes       no       4         3      2     2   

In [5]:
print("describe \n",student_df.describe(include="all"))
print("Sum of null",student_df.isnull().sum())
print("info",student_df.info())

describe 
        school  sex         age address famsize Pstatus        Medu        Fedu   Mjob   Fjob  reason guardian  traveltime   studytime    failures schoolsup famsup paid activities nursery higher internet romantic      famrel    freetime       goout        Dalc        Walc      health    absences          G1          G2          G3
count     395  395  395.000000     395     395     395  395.000000  395.000000    395    395     395      395  395.000000  395.000000  395.000000       395    395  395        395     395    395      395      395  395.000000  395.000000  395.000000  395.000000  395.000000  395.000000  395.000000  395.000000  395.000000  395.000000
unique      2    2         NaN       2       2       2         NaN         NaN      5      5       4        3         NaN         NaN         NaN         2      2    2          2       2      2        2        2         NaN         NaN         NaN         NaN         NaN         NaN         NaN         NaN         NaN      

### 2) Use Chi^2 tests and `stats.chi2_contingency()` to identify:
 - Two pairs of variables that are dependent (are associated with one another).
 - Two pairs of variables that are independent (have no significant relationship).

Does it make intuitive sense why the variables in these pairs might or might not show a relationship to one another? 


![image.png](https://files.slack.com/files-pri/TSZCHB482-F017WLV2UG4/image.png)

##Column describe

In [6]:
student_df["age"].describe()


count    395.000000
mean      16.696203
std        1.276043
min       15.000000
25%       16.000000
50%       17.000000
75%       18.000000
max       22.000000
Name: age, dtype: float64

In [7]:
student_df["absences"].describe()

count    395.000000
mean       5.708861
std        8.003096
min        0.000000
25%        0.000000
50%        4.000000
75%        8.000000
max       75.000000
Name: absences, dtype: float64

In [85]:
student_df["sex"].describe()

count     395
unique      2
top         F
freq      208
Name: sex, dtype: object

In [30]:
student_df["school"].describe()

count     395
unique      2
top        GP
freq      349
Name: school, dtype: object

In [29]:
student_df["traveltime"].describe()

count    395.000000
mean       1.448101
std        0.697505
min        1.000000
25%        1.000000
50%        1.000000
75%        2.000000
max        4.000000
Name: traveltime, dtype: float64

##Sex and absences

In [27]:
student_xtab = pd.crosstab(student_df["sex"],student_df["absences"])
print(student_xtab) #binned categories of the absences, not sure why need to do this step need to ask?
cut_points = [0,22,23,75]
#I am going to cr12,22,23,75] #each pair of numbers is a bin, 0-11, 12-22, 23-75
student_df["absences_categories"] = pd.cut(student_df["absences"],cut_points,labels=['Low','Med','Hi'])
student_xtab = pd.crosstab(student_df["sex"],student_df["absences_categories"])
print(student_xtab)

absences  0   1   2   3   4   5   6   7   8   9   10  11  12  13  14  15  16  17  18  19  20  21  22  23  24  25  26  28  30  38  40  54  56  75
sex                                                                                                                                             
F         63   2  32   2  27   2  19   5   8   1  10   0   8   0   7   2   3   1   2   0   2   1   2   1   1   1   1   1   0   0   1   1   1   1
M         52   1  33   6  26   3  12   2  14   2   7   3   4   3   5   1   4   0   3   1   2   0   1   0   0   0   0   0   1   1   0   0   0   0
absences_categories  Low  Med  Hi
sex                              
F                    136    1   8
M                    133    0   2


In [9]:
female_ab = student_xtab.iloc[0][0:4].values #return the values of a crosstab row as a array of values, use for AI training but not necesary for chi2_contigency
print(female_ab)

[136   1   8]


In [10]:
male_ab = student_xtab.iloc[1][0:4].values #return the values of a crosstab row as a array of values, use for AI training but not necesary for chi2_contigency
print(male_ab)

[133   0   2]


##Chi^2 sex influence absenses

In [12]:
stats.chi2_contingency(student_xtab)

(4.2817758407041815,
 0.11755042125810325,
 2,
 array([[139.30357143,   0.51785714,   5.17857143],
        [129.69642857,   0.48214286,   4.82142857]]))

In [13]:
chi_squared, p_value, dof, expected = stats.chi2_contingency(student_xtab) #so the results are nice and neat
print(f"Chi-Squared: {chi_squared}")
print(f"P-value: {p_value}")
print(f"Degrees of freedon: {dof}")
print("Expected: \n", np.array(expected))
{p_value}

Chi-Squared: 4.2817758407041815
P-value: 0.11755042125810325
Degrees of freedon: 2
Expected: 
 [[139.30357143   0.51785714   5.17857143]
 [129.69642857   0.48214286   4.82142857]]


{0.11755042125810325}

In [26]:
alpha = .05
print("p-value from statistical test is",p_value)
if alpha <= {p_value}.pop():
 print("since \u03B1 <= P_value: we reject the null hypothesis, therefore, the two categorical variables are dependent")
elif alpha > {p_value}.pop():
 print("since \u03B1 > P_value: we fail to reject the null hypothesis, therefore, the two categorical variables are independent")

p-value from statistical test is 0.11755042125810325
since α <= P_value: we reject the null hypothesis, therefore, the two categorical variables are dependent


##Chi^2 sex influence travel time

In [21]:
student_tt = pd.crosstab(student_df["school"],student_df["traveltime"])
print(student_tt) #binned categories of the traveltime, not sure why need to do this step need to ask?
"The dataset providers made binned categorical: traveltime - home to school travel time (numeric: 1 - <15 min., 2 - 15 to 30 min., 3 - 30 min. to 1 hour, or 4 - >1 hour)"
student_tt.columns = ["shortest", "short", "moderate", "long"]
print(student_tt)

traveltime    1   2   3  4
school                    
GP          243  84  15  7
MS           14  23   8  1
        shortest  short  moderate  long
school                                 
GP           243     84        15     7
MS            14     23         8     1


In [22]:
stats.chi2_contingency(student_tt)

(31.656123714485112,
 6.183726050805027e-07,
 3,
 array([[227.07088608,  94.53924051,  20.32151899,   7.06835443],
        [ 29.92911392,  12.46075949,   2.67848101,   0.93164557]]))

In [28]:
alpha = .05
print("p-value from statistical test is",p_value)
if alpha <= stats.chi2_contingency(student_tt)[1]:
 print("since \u03B1 <= P_value we reject the null hypothesis, therefore, the two categorical variables are dependent")
elif alpha > stats.chi2_contingency(student_tt)[1]:
 print("since \u03B1 > P_value, we fail to reject the null hypothesis, therefore, the two categorical variables are independent")

p-value from statistical test is 0.11755042125810325
since α > P_value, we fail to reject the null hypothesis, therefore, the two categorical variables are independent


#3) Use NumPy to perform your own chi^2 test "from scratch" 

Pick any of the chi^2 tests that you ran in part 2 and try them on your own. You should get the same results that Scipy got for all four values returned from `chi2_contingency()`

In [55]:
student_xtab2 = pd.crosstab(student_df["sex"],student_df["absences"])
print(student_xtab2) #binned categories of the absences, not sure why need to do this step need to ask?
cut_points = [0,22,23,75]
#I am going to cr12,22,23,75] #each pair of numbers is a bin, 0-11, 12-22, 23-75
student_df["absences_categories"] = pd.cut(student_df["absences"],cut_points,labels=['Low','Med','Hi'])
student_xtab2 = pd.crosstab(student_df["sex"],student_df["absences_categories"], margins=True) #when doing it by hand need the margins rows and columns
print(student_xtab2)

absences  0   1   2   3   4   5   6   7   8   9   10  11  12  13  14  15  16  17  18  19  20  21  22  23  24  25  26  28  30  38  40  54  56  75
sex                                                                                                                                             
F         63   2  32   2  27   2  19   5   8   1  10   0   8   0   7   2   3   1   2   0   2   1   2   1   1   1   1   1   0   0   1   1   1   1
M         52   1  33   6  26   3  12   2  14   2   7   3   4   3   5   1   4   0   3   1   2   0   1   0   0   0   0   0   1   1   0   0   0   0
absences_categories  Low  Med  Hi  All
sex                                   
F                    136    1   8  145
M                    133    0   2  135
All                  269    1  10  280


In [32]:
female_ab = student_xtab2.iloc[0][0:4].values #return the values of a crosstab row as a array of values, use for AI training but not necesary for chi2_contigency
print(female_ab)

[136   1   8]


In [33]:
male_ab = student_xtab2.iloc[1][0:4].values #return the values of a crosstab row as a array of values, use for AI training but not necesary for chi2_contigency
print(male_ab)

[133   0   2]


In [44]:
from IPython.display import display, Image
url = 'https://media.cheggcdn.com/study/ab5/ab55a54b-7490-4847-90f2-ca829fa978ec/CL-353V1.png'
chi2 = Image(url=url, width=600)
display(chi2)

In [76]:
# Expected Frequency
row_sums = student_xtab2.iloc[0:2, 3].values #I am returning the totals of row values, which are in the column named "All"
col_sums = student_xtab2.iloc[2,0:3].values #I am returning the totals of column values, which are in the row named "All"
print(row_sums)
print(col_sums)

[145 135]
[269   1  10]


In [79]:
#sum of all row line from the crosstab table
total = student_xtab2.loc["All",'All']
print(total)

280


In [90]:
expected = [] #this is use to get the expected values of the chi-square, chi2_contingency()
for i in range(len(row_sums)):
  expected_row = []
  for column in col_sums:
    expected_val = column*row_sums[i]/total
    expected_row.append(expected_val)
  expected.append(expected_row)
print(expected)
expected = np.array(expected) #We want to convert this into a np.array so we can use the arithmetic ability of pandas.
print(expected.shape)
print(expected)

[[139.30357142857142, 0.5178571428571429, 5.178571428571429], [129.69642857142858, 0.48214285714285715, 4.821428571428571]]
(2, 3)
[[139.30357143   0.51785714   5.17857143]
 [129.69642857   0.48214286   4.82142857]]


##Observed values

In [87]:
observed = pd.crosstab(student_df["sex"],student_df["absences_categories"]).values #recreating crostab to return all individual values, without the margins, it think iloc will work as well.
print(observed.shape)
print(observed)

(2, 3)
[[136   1   8]
 [133   0   2]]


In [94]:
chi_squared_2 =((observed-expected)**2/(expected)).sum() #I call it chi_squared_2 because I have use chi_square before that variable before on this colab
print(type(((observed-expected)**2/(expected)).sum()))
print(f"Chi-Squared:{chi_squared_2}") #this is the chi_squared value

<class 'numpy.float64'>
Degrees of freedom:4.2817758407041815


In [95]:
#Degrees of freedom of a chi-squared test
#get it for both the columns and rows
dof_2 = (len(row_sums)-1)*(len(col_sums)-1) #I have use dof variable previously so use ..._2 instead
print(f"Degrees of freedom: {dof_2}") #results is basically daframeshape(2,3) ((2-1)*(3-1))

Degrees of freedom: 2


## Stretch goals:

### 1. Refactor your code so it is elegant, readable, and holds reusable code in functions.

In [16]:
# YOUR WORK HERE



### 2. Check For Understanding - Study and write your own explanations/definitions for these topics:

- What is a sample "estimate" in statistics?

- What are hypothesis test? How are they useful?

- What is a "null hypothesis?"

- What is a p-value and what does it represent?

- What does it mean for something to be "statistically significant?"

- What is a test statistic and how does it relate to a p-value?

- What are "degrees of freedom" and how are they calculated in a 1-sample, 2-sample, and chi^2 test for independence?

## Resources

- [Interactive visualize the Chi-Squared test](https://homepage.divms.uiowa.edu/~mbognar/applets/chisq.html)
- [Calculation of Chi-Squared test statistic](https://en.wikipedia.org/wiki/Pearson%27s_chi-squared_test)
- [Visualization of a confidence interval generated by R code](https://commons.wikimedia.org/wiki/File:Confidence-interval.svg)
- [Expected value of a squared standard normal](https://math.stackexchange.com/questions/264061/expected-value-calculation-for-squared-normal-distribution) (it's 1 - which is why the expected value of a Chi-Squared with $n$ degrees of freedom is $n$, as it's the sum of $n$ squared standard normals)